In [ ]:
import pandas as pd

## My API key  & library_Id are stored in another file
api_key,library_id = pd.read_json("Zotero_API_key.json", typ='series')

library_type="group"




from pyzotero import zotero
zot = zotero.Zotero(library_id, library_type, api_key)

ID_Universities='EWSYI3RS' # to access the files in the university subfolder.

In [ ]:
for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")

In [ ]:
def relevantItemInfo(dictionary):    
    if dictionary["itemType"]=='attachment':
        # if directory --> key leads to the correct folder
        
        relevant_keys=['key','url']
        newDict={key: dictionary[key] for key in relevant_keys}
        #print(dictionary)
        newDict["note"]=None
       # print(newDict)
        return newDict
    
    if dictionary["itemType"]=="note":
        relevant_keys=['key','note']
        newDict={key: dictionary[key] for key in relevant_keys}
        newDict["url"]=None
        return newDict

    

for i in zot.collection_items(key)[:2]:
    dictionary=i["data"]
    print(dictionary, end="\n\n")
    
    Info=relevantItemInfo(dictionary)
    print(Info, end="\n\n")

In [ ]:
dictList=[]

for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")
    
    for i in zot.collection_items(key):
        # only use data if there is an attachment.
        if i["data"]["itemType"]=='attachment':
            Info=relevantItemInfo(i["data"])
            dictList+=[{"Uni": name,"FileKey":Info["key"],"url":Info["url"]}]
    


In [ ]:
df=pd.DataFrame(dictList)
df.head()

In [ ]:
def FileLinks(Filekey):
   #print(Filekey)
    import os
    DIR=f"C:\\Users\\mr\\Zotero\\storage\\{Filekey}"
    #print(DIR)
    FileNames=os.listdir(DIR)
    file=[i for i in FileNames if i.endswith(".html")]
    return DIR+"\\"+FileNames[-1]



df["file"]=df.FileKey.apply(FileLinks)
df.head()

In [ ]:
df.to_json("2023-05-10_Uni_Files_raw.json")

In [ ]:
df=pd.read_json("2023-05-10_Uni_Files_raw.json")

# Add Content from HTMLs

In [ ]:
from htmldate import find_date
#find_date("https://www.uu.nl/en/events/chat-gpt-discover-the-magic-of-ai")

def find_dateX(x):
    try:
        return find_date(x)
    except:
        print(x)


df["date"]=df.url.apply(find_dateX)

In [ ]:
import MyLib.nlp as nlp
import MyLib.HTML_prep as HTML_prep
import MyLib.analysis as analysis 
%load_ext autoreload
%autoreload 2

In [ ]:

def FileInfo(FilePath):
    Title,h1,h2,text,linkName,linkUrl=None,None,None,None,None,None
    if FilePath.endswith(".html"):
    
        try:
            #print(FilePath)

            with open(FilePath,"r", encoding='utf-8') as f:
                text= f.read()

            from bs4 import BeautifulSoup

            soup = BeautifulSoup(text, 'html.parser')
            
            Title=soup.title.text
            
            #soup=HTML_prep.removeIMGs(soup,IMG_filename=Title.replace(" ","_"))      


            All_divs=[p.parent for p in soup("p") if p.parent.name=="div"]
            All_divs=(list(set(All_divs)))
            All_divs=[HTML_prep.removeIMGs(div,no_img=True,IMG_filename=Title.replace(" ","_")) for div in All_divs]        

            h1=[item for sublist in [[i.text for i in div("h1")] for div in All_divs if div("h1")!=None] for item in sublist]
            h2=[item for sublist in [[i.text for i in div("h2")] for div in All_divs if div("h2")!=None] for item in sublist]
            text=[item for sublist in [[i.text for i in div("p")] for div in All_divs if div("p")!=None] for item in sublist]
            linkName=[item for sublist in [[i.text for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]
            linkUrl=[item for sublist in [[i.get('href') for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]


        except:
            print(f"error with: {FilePath}",end=". ")
    return pd.Series([Title,h1,h2,text,linkName,linkUrl])
        
        

#df.file[2:6].apply(FileInfo)
df[["Title","h1","h2","text","linkName","linkUrl"]]=df.file.apply(FileInfo)


In [ ]:
df.to_json("2023-05-10_Uni_Files_NLP.json")

In [ ]:
def Insert_title(text,Title):
    if Title!=None and type(text)==list:
        text=[Title]+text
        return text
    else:
        return text


df["text"]=df[["text","Title"]].apply(lambda x: Insert_title(*x),axis=1)

In [ ]:
ChatGPT_Terms="ChatGPT, GPT3, GPT4, OpenAI, AI, LLM, A.I, GPT, Chatbot, technology, tool, app, Artificial Intelligence, Large Language Models"
ChatGPT_Terms=ChatGPT_Terms.split(", ")
ChatGPT_Terms=[i.lower() for i in ChatGPT_Terms]


In [ ]:
def GPTparagraphs(p_list, by=ChatGPT_Terms):
    if p_list!=None:
        AI_Content=[]
        for text in p_list:
            AI_Content_Binary=nlp.WordlistFilter(text, by)
            if AI_Content_Binary==True:
                AI_Content.append(text)
        return AI_Content
            


df["AI_paragraphs"]=df.text.apply(GPTparagraphs)



In [ ]:
df2=df[df.file.apply(lambda x: x.endswith("html"))]
df2.text=df2["AI_paragraphs"].apply(lambda x:" ".join(x))

In [ ]:
df2[df2.text.apply(lambda x: type(x))!=str]

In [ ]:
df2["LEN"]=df2.text.apply(lambda x: len(x))
df2=df2[df2.LEN<df2.LEN.max()] ###Cut out the very large list

In [ ]:
df2.LEN.describe()

In [ ]:
df3=nlp.NLP_Pipeline(df2, sentiment=False, language="en",translate=True)

In [ ]:
df3.to_json("2023-05-10_Zotero_AI_nlp_en.json")

In [ ]:
Lists=df3.NoStopwords.to_list()
Lists=[i for i in Lists if i!=None]
Words=[item for sublist in Lists for item in sublist]

from collections import Counter
a_counter = Counter(Words)
most_common = a_counter.most_common(10)

most_common

In [ ]:
from ipywidgets import interactive, interact

@interact(column=["NoStopwords"],Mostcommon=(5,200))
def make_wordcloud(column="NoStopwords",Mostcommon=20,Save_as=""):
    hashtags=[i.lower() for s in df3[column].dropna() for i in s] # hashtags OR # no stopwords
    analysis.make_wordcloud(hashtags,filename=Save_as,Mostcommon=Mostcommon)

In [ ]:
df3.to_excel("2023-05-08_ZoteroHTMLs.xlsx")